In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from IPython.display import display, HTML

def read_flow(sender_file_name, receiver_file_name):
    s = pd.read_csv(sender_file_name, sep='\t')
    r = pd.read_csv(receiver_file_name, sep='\t')
    # print(sender_file_name)
    df = pd.merge(s, r, how='outer', on='pkt_id')
    df['latency [s]'] = df['rcv_t [s]'] - df['snd_t [s]']
    df['latency [ms]'] = (df['rcv_t [s]'] - df['snd_t [s]']) * 1000
    df['sec'] = df['rcv_t [s]'] - df.at[0, 'snd_t [s]']
    df['disk_sec'] = df['sec'].apply(np.floor)
    df['disk_msec'] = df['sec'].apply(lambda x: np.floor(x * 1000))
    return df

def tp_array(df, bucket_size_ms=100):
    m = int(df['disk_msec'].max() / bucket_size_ms)
    byt, borders, _ = stats.binned_statistic(df['disk_msec'], df['payload [bytes]'], 'sum', bins=[i*bucket_size_ms for i in range(m)])
    ndf = pd.DataFrame({'msec': borders[:-1], 'tp [Mbps]': byt * 0.008 / bucket_size_ms})
    return ndf

def mean_tp(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = tp_array(df)
    df = df[(df['msec'] >= cutoff_s*1e3) & (df['msec'] < latest_time - cutoff_s*1e3)]
    return df['tp [Mbps]'].mean()

def mean_latency(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = df[(df['disk_msec'] > cutoff_s * 1000) & (df['disk_msec'] < latest_time - cutoff_s * 1000)]
    return df['latency [ms]'].mean()


topologies = ['mptcp-host-pair', 'MPflow_lpkt','MPvsSP', 'MPvsSP_lpkt']
congestioncontr = ['lia', 'olia', 'balia', 'wvegas']